# Deploy an MLflow model with SageMaker

## Install MLflow

In [1]:
!pip install -q mlflow==2.0.1

## Setup environment

In [3]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston
from mlflow.deployments import get_deploy_client

# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = sagemaker.get_execution_role() 
# uri of your remote mlflow server
tracking_uri = 'http://mlflo-mlflo-110tkpke38vv-e1e1dc1c5f95c722.elb.us-east-1.amazonaws.com/' 
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

## Build MLflow docker image to serve the model with SageMaker 

In [8]:
!mlflow sagemaker build-and-push-container

2023/02/18 07:23:04 INFO mlflow.models.docker_utils: Building docker image with name mlflow-pyfunc
Sending build context to Docker daemon  4.096kB

Step 1/26 : FROM ubuntu:20.04
 ---> e40cf56b4be3
Step 2/26 : RUN apt-get -y update
 ---> Using cache
 ---> 0b7c2dab84e2
Step 3/26 : RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get install -y --no-install-recommends          wget          curl          nginx          ca-certificates          bzip2          build-essential          cmake          openjdk-8-jdk          git-core          maven     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 281cf3685e7f
Step 4/26 : RUN apt -y update
 ---> Using cache
 ---> ff61fea9b084
Step 5/26 : RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get -y install tzdata
 ---> Using cache
 ---> 8d7fc5d7e3e1
Step 6/26 : RUN apt-get install -y     libssl-dev zlib1g-dev libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm     libncursesw5-dev xz-utils tk-dev libxml2-dev libxmlsec1-dev libffi-dev

1feeb3ed: Pushing  448.6MB/931.3MB

c803ce8: Pushed   903.6MB/890.1MB2.0.1: digest: sha256:3cc3f861a3e08c7dee262f1b9987fb7eabdc6b4da52904af279feecc93b3f4e9 size: 4530


In [28]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '902607334202.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:2.0.1'

## Deploy a SageMaker endpoint with our scikit-learn model

In [29]:
endpoint_name = 'boston-housing'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/mlflow-v1/1'

In [30]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region
}

client = get_deploy_client("sagemaker")

client.create_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

2023/02/18 07:56:17 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2023/02/18 07:56:17 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2023/02/18 07:56:17 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-1-902607334202` already exists. Skipping creation.
2023/02/18 07:56:18 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': 'FWJK04DZBQKPEAD8', 'HostId': 'sBQZD5xMHFvCqg/3HF0aErjlwPyz+LdO0ru5hgK6DHGKhF773yhsD547yiP0WsKw0eWYWPS7Cf8=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'sBQZD5xMHFvCqg/3HF0aErjlwPyz+LdO0ru5hgK6DHGKhF773yhsD547yiP0WsKw0eWYWPS7Cf8=', 'x-amz-request-id': 'FWJK04DZBQKPEAD8', 'date': 'Sat, 18 Feb 2023 07:56:19 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2023/02/18 07:56:18 INFO mlflow.sagemaker: Creating new endpoint with name: boston-housing ...
2023/02/18 07:56:18 INFO mlflow.sagemaker: Created model with arn: arn:aws:sagemaker:us-east-1:

{'name': 'boston-housing', 'flavor': 'python_function'}

## Predict

In [31]:
# load boston dataset
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [33]:
client = get_deploy_client(f"sagemaker:/{region}")

payload = df.iloc[[0]]
prediction = client.predict(endpoint_name, df.iloc[[0]])

print(f'Payload: {payload}')
print(f'Prediction: {prediction}')

Payload:       CRIM    ZN  INDUS  CHAS    NOX     RM   AGE   DIS  RAD    TAX  PTRATIO  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.09  1.0  296.0     15.3   

       B  LSTAT  
0  396.9   4.98  
Prediction: {'predictions': [28.638372253369607]}


## Delete endpoint

In [34]:
client.delete_deployment(endpoint_name, config=config)

2023/02/18 08:23:37 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:us-east-1:902607334202:endpoint/boston-housing
2023/02/18 08:23:37 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2023/02/18 08:23:37 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2023/02/18 08:23:42 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2023/02/18 08:23:42 INFO mlflow.sagemaker: Cleaning up unused resources...
2023/02/18 08:23:42 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:us-east-1:902607334202:endpoint-config/boston-housing-config-dubng075tfwyoabaguugjxw
2023/02/18 08:23:42 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:us-east-1:902607334202:model/boston-housing-model-bdyfweabcrae0p59u0y31yw
